In [1]:
import pandas as pd
import numpy as np
import glob
import csv
import os
import re

In [2]:
data_folder = 'experiments2023'
# find all csv files in the data folder and its subfolders
data_file_list = glob.glob(data_folder + '/**/*.csv', recursive=True)
print(data_file_list)

['experiments2023\\cpu\\client_1_cpu_1000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_1_cpu_2000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_1_cpu_3000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_1_cpu_4000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_1_cpu_5000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_1_cpu_6000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_3_cpu_1000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_3_cpu_100m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_3_cpu_2000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_3_cpu_3000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_3_cpu_4000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_3_cpu_5000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_3_cpu_6000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_5_cpu_1000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_5_cpu_2000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\client_5_cpu_3000m_mem_16_gpu_0.csv', 'experiments2023\\cpu\\c

In [3]:
'''
format of data_points:
data_points = {
    'cpu': num_cpu,
    'mem': num_mem,
    'gpu': num_gpu,
    'gou_mem': num_gpu_mem,
    'client': num_client,
    'time_extract_network_package': extract_network_package_time,
    'time_convert_data_to_numpy': convert_data_to_numpy_time,
    'time_convert_numpy_to_opencv': convert_numpy_to_opencv_time,
    'time_model_inference': model_inference_time,
}
'''
data_points = [] # list of data points

# regex for extracting data from log
# try to match pattern
# ts=12345
re_operation_times = re.compile(r'ts=(\d+)')
# value_string='xxxxx'
re_operation_name = re.compile(r'value_string=\'(([a-z]|[A-Z]|[0-9]|\ )*)\'')

for data_file_path in data_file_list:
    data_point_setup = {}
    # parse file name
    file_name = os.path.basename(data_file_path).split('.')[0]
    print(file_name)
    file_name_split = file_name.split('_')
    # get number of clients
    data_point_setup['client'] = int(file_name_split[file_name_split.index('client')+1])
    # get gpu and gpu memory
    if 'gpu' in file_name_split:
        gpu_config = file_name_split[file_name_split.index('gpu')+1]
        if gpu_config == '0':
            data_point_setup['gpu'] = 0
            data_point_setup['gpu_mem'] = 0
        else:
            gpu_config_split = gpu_config.split('g')
            data_point_setup['gpu'] = int(gpu_config_split[0])
            data_point_setup['gpu_mem'] = int(gpu_config_split[1])
    else:
        data_point_setup['gpu'] = 0
        data_point_setup['gpu_mem'] = 0
    # get cpu
    cpu_str = file_name_split[file_name_split.index('cpu')+1]
    if 'm' in cpu_str:
        cpu_str = cpu_str.split('m')[0]
        data_point_setup['cpu'] = int(cpu_str)
    else:
        data_point_setup['cpu'] = int(cpu_str) * 1000
    # get mem
    if 'mem' in file_name_split:
        mem_str = file_name_split[file_name_split.index('mem')+1]
    elif 'memory' in file_name_split:
        mem_str = file_name_split[file_name_split.index('memory')+1].split('g')[0]
    else:
        mem_str = '0'
    data_point_setup['mem'] = int(mem_str)

    # read csv file
    with open(data_file_path, 'r') as f:
        reader = csv.reader(f)
        for id, row in enumerate(reader):
            if row[0] == 'detect':
                data_point = {
                    'cpu': data_point_setup['cpu'],
                    'mem': data_point_setup['mem'],
                    'gpu': data_point_setup['gpu'],
                    'gpu_mem': data_point_setup['gpu_mem'],
                    'client': data_point_setup['client']
                }
                logs_str = row[3]
                operation_time_list = re_operation_times.findall(logs_str)
                operation_name_list = re_operation_name.findall(logs_str)
                time_extract_network_package = int(operation_time_list[1]) - int(operation_time_list[0])
                time_convert_data_to_numpy = int(operation_time_list[2]) - int(operation_time_list[1])
                time_convert_numpy_to_opencv = int(operation_time_list[3]) - int(operation_time_list[2])
                time_model_inference = int(operation_time_list[4]) - int(operation_time_list[3])
                data_point['time_extract_network_package'] = time_extract_network_package
                data_point['time_convert_data_to_numpy'] = time_convert_data_to_numpy
                data_point['time_convert_numpy_to_opencv'] = time_convert_numpy_to_opencv
                data_point['time_model_inference'] = time_model_inference
                data_points.append(data_point)
    

client_1_cpu_1000m_mem_16_gpu_0
client_1_cpu_2000m_mem_16_gpu_0
client_1_cpu_3000m_mem_16_gpu_0
client_1_cpu_4000m_mem_16_gpu_0
client_1_cpu_5000m_mem_16_gpu_0
client_1_cpu_6000m_mem_16_gpu_0
client_3_cpu_1000m_mem_16_gpu_0
client_3_cpu_100m_mem_16_gpu_0
client_3_cpu_2000m_mem_16_gpu_0
client_3_cpu_3000m_mem_16_gpu_0
client_3_cpu_4000m_mem_16_gpu_0
client_3_cpu_5000m_mem_16_gpu_0
client_3_cpu_6000m_mem_16_gpu_0
client_5_cpu_1000m_mem_16_gpu_0
client_5_cpu_2000m_mem_16_gpu_0
client_5_cpu_3000m_mem_16_gpu_0
client_5_cpu_4000m_mem_16_gpu_0
client_5_cpu_6000m_mem_16_gpu_0
client_3_cpu_6000m_mem_16_gpu_0
client_3_cpu_6000m_mem_16_gpu_1g5gb
client_3_cpu_6000m_mem_16_gpu_2g10gb
client_3_cpu_6000m_mem_16_gpu_3g20gb
client_3_cpu_6_mem_16_gpu_1g5gb
client_5_memory_16g_cpu_6_data_points
client_5_memory_16g_cpu_6_data_points_2
client_1_memory_16g_cpu_6_data_points
client_2_memory_16g_cpu_6_data_points
client_3_memory_16g_cpu_6_data_points
client_4_memory_16g_cpu_6_data_points
client_5_memory_16g_c

In [4]:
# output data to csv file
df = pd.DataFrame(data_points)
df.to_csv('data_points.csv', index=False)